In [1]:
USE_4BIT = False 

In [2]:
# ╔══════════════════════════════════════════════════════════╗
# ║ CELL 0 — CONFIG                                          ║
# ╚══════════════════════════════════════════════════════════╝
FOLDER_ID   = "/kaggle/input/llm-summaries"      # path that Kaggle shows after upload
MODEL_ID    = "meta-llama/Meta-Llama-3-8B-Instruct"
USE_4BIT    = False          # True ➜ needs bitsandbytes install, VRAM ≈ 5 GB
ROWS_LIMIT  = None           # e.g. 50 for a quick smoke-test, or None for all rows


In [3]:
# ╔══════════════════════════════════════════════════════════╗
# ║ CELL 1 — Install deps & (optionally) bitsandbytes        ║
# ╚══════════════════════════════════════════════════════════╝
!pip -q install "transformers>=4.40" accelerate sentencepiece tqdm openpyxl

if USE_4BIT:
    # Turn Internet ON for 30 s, then run this cell. Switch back OFF afterwards.
    !pip -q install --no-cache-dir "bitsandbytes==0.42.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00:00:0100:01


In [4]:
# ╔══════════════════════════════════════════════════════════╗
# ║ CELL 2 — Hugging Face login & cache into working dir     ║
# ╚══════════════════════════════════════════════════════════╝
from huggingface_hub import login
login()                                     # paste your READ token

import os, pathlib
os.environ["HF_HOME"] = "/kaggle/working/hf_cache"
pathlib.Path(os.environ["HF_HOME"]).mkdir(parents=True, exist_ok=True)



In [5]:
# ╔══════════════════════════════════════════════════════════╗
# ║ CELL 3 — Load LLAMA-3 (4-bit or fp16)                    ║
# ╚══════════════════════════════════════════════════════════╝
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig, pipeline)
import torch

tok = AutoTokenizer.from_pretrained(MODEL_ID)

if USE_4BIT:
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_cfg,
        trust_remote_code=True
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.float16,    # full precision, ~14 GB VRAM
        trust_remote_code=True
    )

gen = pipeline("text-generation",
               model=model,
               tokenizer=tok,
               max_new_tokens=512,
               do_sample=False)
               



2025-06-01 07:44:34.373830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748763874.590012      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748763874.664489      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
# ╔══════════════════════════════════════════════════════════╗
# ║ CELL 4 — Helper: ½-length summary                        ║
# ╚══════════════════════════════════════════════════════════╝
import math
import pandas as pd
from tqdm.notebook import tqdm

# Define the summarization function
def summarise_half(text: str) -> str:
    tgt = max(30, math.ceil(len(text.split()) * 0.5))
    prompt = tok.apply_chat_template(
        [
            {"role": "system",
             "content": "You are a helpful legal assistant. Create concise, accurate summaries."},
            {"role": "user",
             "content": f"Summarise the following legal text in about {tgt} words:\n\n{text}"}
        ],
        tokenize=False,
        add_generation_prompt=True
    )
    raw_output = gen(prompt, max_new_tokens=tgt * 2)[0]["generated_text"]
    
    # Clean the output
    if 'assistant' in raw_output.lower():
        result = raw_output.lower().split('assistant', 1)[-1].strip()
    else:
        result = raw_output.strip()
    
    unwanted_starts = [
        "Here is a concise summary of the legal text:",
        "Sure,", "Certainly,", "Here is", "Summary:", "As a legal assistant",
    ]
    for prefix in unwanted_starts:
        if result.lower().startswith(prefix.lower()):
            result = result[len(prefix):].strip(": \n")
    
    result = result.strip().replace('\n', ' ').replace('\r', '').strip()
    return result

In [7]:
# ╔══════════════════════════════════════════════════════════╗
# ║ CELL 5 — Build half-only Excel file                      ║
# ╚══════════════════════════════════════════════════════════╝
def build_half_only(src_path: str, col_name: str, out_path: str):
    df = pd.read_excel(src_path)
    if ROWS_LIMIT:
        df = df.head(ROWS_LIMIT)  # Optional: limit rows for testing
    
    texts = df[col_name].fillna("").astype(str)
    
    # Summarize each text
    halves = [ summarise_half(t) for t in tqdm(texts, desc=f"½ summaries for {col_name}") ]
    
    # Save results to Excel
    result_df = pd.DataFrame({
        f"full_{col_name.lower()}": texts,
        f"summary_half_{col_name.lower()}": halves
    })
    result_df.to_excel(out_path, index=False)
    print("✅  Saved →", out_path)


In [8]:
OUT = "/kaggle/working"
import pandas as pd
from tqdm.notebook import tqdm

# Load the full dataset
df_src = pd.read_excel(f"{FOLDER_ID}/dataset_description_only.xlsx")
texts = df_src["Description"].fillna("").astype(str)

# Select rows from 992 to the end
texts_remaining = texts.iloc[600:1200]
print(f"Processing {len(texts_remaining)} rows (from 992 to {len(texts)}).")

# Summarize remaining rows
new_summaries = [summarise_half(txt) for txt in tqdm(texts_remaining, desc="Summarizing remaining rows")]

# Save to new Excel file
df_remaining = pd.DataFrame({
    "full_description": texts_remaining,
    "summary_half_description": new_summaries
})

df_remaining.to_excel(f"{OUT}/summary_llama_description_p1.xlsx", index=False)
print("✅ Saved remaining summaries.")

Processing 600 rows (from 992 to 1200).


Summarizing remaining rows:   0%|          | 0/600 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_

✅ Saved remaining summaries.


In [ ]:
FOLDER_ID   = "/kaggle/input/llm-summaries"  # Change this to your dataset folder
MODEL_ID    = "meta-llama/Meta-Llama-3-8B-Instruct"
USE_4BIT    = False          # Set True if using bitsandbytes for 4-bit quantization
ROWS_LIMIT  = None           # Set to a number for testing, or None to process all rows
INPUT_FILE  = f"{FOLDER_ID}/dataset_description_only.xlsx"
OUTPUT_FILE = "/kaggle/working/summary_for_classification.xlsx"
COLUMN_NAME = "Description"